In [10]:
import pandas as pd
import numpy as np

In [11]:
df = pd.read_excel('Disbursed_cases_DS_output.xlsx', sheet_name='Scores')

In [12]:
df.head()

,unique_id,full_name,pin_code,phone_number,pan_number,email,DS_Score,Name (Bank),Name (PAN),DOB,...,microsoft (Phone),paytm (Phone),shaadi (Phone),skype (Phone),swiggy (Phone),toi (Phone),whatsapp (Phone),yatra (Phone),zoho (Phone),Black List Match
0,100,NAKUL GANDHI,110034,9968610064,BLLPG1720J,nakul@gasyparcel.co.in,590,YELLOW STAR,NAKUL GANDHI,1991-05-28,...,False,True,False,False,True,False,True,False,False,False
1,101,DEEPAK DEEPAK KUMAR,110020,7088551995,HPOPK4658L,noemail@noemail.com,620,DEEPAK KUMAR,DEEPAK KUMAR,1995-07-18,...,False,True,False,False,True,True,True,False,False,False
2,102,SACHIN BALIRAM AUCHAR,400043,9892429271,BPIPA9897M,sachinbalramauchar1@gmail.com,760,Sachin Baliram Auchar,SACHIN BALIRAM AUCHAR,1995-05-05,...,False,True,False,False,True,False,True,False,False,False
3,103,ASHISHKUMAR BHARATBHAI TUNDAVIYA,380006,8487006223,BETPT6620P,ashishtundaviya@gmail.com,770,TUNDAVIYA ASHISHKUMAR BHARATBHAI,ASHISHKUMAR BHARATBHAI TUNDAVIYA,1994-10-20,...,False,True,False,False,False,False,True,False,False,False
4,104,CONELAKANCHUVISHAL VISHAL,502032,9290939596,BOIPC4723F,vishal@gmail.com,770,Conelakanchu Vishal,VISHAL CONELAKANCHU,1997-01-13,...,False,True,False,False,True,True,True,False,False,False


In [13]:
# Calculate deciles
deciles = np.percentile(df['DS_Score'].round(2), np.arange(0, 101, 10))

# Assigning labels to deciles based on endpoints
labels = [f"{float(deciles[i])}-{float(deciles[i+1])}" for i in range(len(deciles)-1)]

# Creating deciles column
df['DS_Score_bkt'] = pd.qcut(df['DS_Score'].round(2), q=10, labels=labels)

In [14]:
master_data = pd.read_excel('master_data_Feb_1902.xlsx')

In [15]:
df = pd.merge(df, master_data, on='phone_number', how='left')

In [16]:
df['isAny (Email)'] = np.where(df[[x for x in df.columns if "(Email)" in x]].isna().sum(axis=1)>0,np.nan,
    np.where(df[[x for x in df.columns if "(Email)" in x]].sum(axis=1) > 0,1,0))

In [17]:
payu_data = pd.read_excel('/Users/keyurpethad/Library/CloudStorage/GoogleDrive-keyur@niro.money/My Drive/PayU Scrub/niromoney_combined_scores_external.xlsx', sheet_name='data')
df = pd.merge(df, payu_data[['hashed_id', 'Payu Hit', 'stl_a_score', 'stl_a_score_cut']], left_on='hashed_phone', right_on='hashed_id', how = 'left')

In [20]:
df["Feb'24_Bounced_flag"] = np.where(df["Feb'24 Presentation"].isin(['Bounced', 'Bounced (DPD)', 'NPA', 'NPA (unsettled)']),1,0)

In [25]:
df["Feb'24 Presentation"].value_counts(dropna=False)

Feb'24 Presentation
Cleared            1544
FORECLOSED          388
Closed              315
Bounced             310
NPA (unsettled)     245
Bounced (DPD)       162
Advance             161
NPA                  74
T-BOUNCE             17
Settled              10
NaN                   1
Name: count, dtype: int64

In [22]:
df.to_excel('datasutram_pred_master2.xlsx')